<a href="https://colab.research.google.com/github/brucewuquant/pythonstuff/blob/master/quantlib_convertible.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install QuantLib-Python

     |████████████████████████████████| 16.9MB 19.1MB/s 


In [0]:
import QuantLib as ql

In [0]:
calculation_date = ql.Date(28,5,2019)

In [0]:
ql.Settings.instance().evaluationDate = calculation_date

Conversion_Ratio = Redemption_Amount / Conversion_Price

In [0]:
redemption = 100.00
face_amount = 100.0
spot_price = 29.04
conversion_price = 26.0
conversion_ratio = 3.84615

issue_date = ql.Date(15,3,2016)
maturity_date = ql.Date(15,3,2022)

settlement_days = 2
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
coupon = 0.0575
frequency = ql.Semiannual
tenor = ql.Period(frequency)

day_count = ql.Thirty360()
accrual_convention = ql.Unadjusted
payment_convention = ql.Unadjusted

call_dates = [ql.Date(20, 3, 2020)]
call_price = 100.0
put_dates = [ql.Date(20,3,2020),ql.Date(15,4,2021)]
put_price=100.0

In [0]:
# assumptions
dividend_yield = 0.02
credit_spread_rate = 0.03  
risk_free_rate = 0.04
volatility = 0.40

In [0]:
callability_schedule = ql.CallabilitySchedule()

for call_date in call_dates:
  callability_price = ql.CallabilityPrice(call_price, ql.CallabilityPrice.Clean)
  callability_schedule.append(ql.Callability(callability_price, 
                                            ql.Callability.Call,
                                            call_date))

In [0]:
for put_date in put_dates:
    puttability_price = ql.CallabilityPrice(put_price, 
                                            ql.CallabilityPrice.Clean)
    callability_schedule.append(ql.Callability(puttability_price,
                                               ql.Callability.Put,
                                               put_date))

In [0]:
dividend_schedule = ql.DividendSchedule() # No dividends
dividend_amount = dividend_yield*spot_price
next_dividend_date = ql.Date(1,12,2004)
dividend_amount = spot_price*dividend_yield
for i in range(4):
    date = calendar.advance(next_dividend_date, 1, ql.Years)
    dividend_schedule.append(
        ql.FixedDividend(dividend_amount, date)
    )

In [0]:
schedule = ql.Schedule(issue_date, maturity_date, tenor,
                       calendar, accrual_convention, accrual_convention,
                       ql.DateGeneration.Backward, False)

credit_spread_handle = ql.QuoteHandle(ql.SimpleQuote(credit_spread_rate))
exercise = ql.AmericanExercise(calculation_date, maturity_date)

convertible_bond = ql.ConvertibleFixedCouponBond(exercise,
                                                 conversion_ratio,
                                                 dividend_schedule,
                                                 callability_schedule, 
                                                 credit_spread_handle,
                                                 issue_date,
                                                 settlement_days,
                                                 [coupon],
                                                 day_count,
                                                 schedule,
                                                 redemption)

In [0]:
spot_price_handle = ql.QuoteHandle(ql.SimpleQuote(spot_price))
yield_ts_handle = ql.YieldTermStructureHandle(
    ql.FlatForward(calculation_date, risk_free_rate, day_count)
)
dividend_ts_handle = ql.YieldTermStructureHandle(
    ql.FlatForward(calculation_date, dividend_yield, day_count)
)
volatility_ts_handle = ql.BlackVolTermStructureHandle(
    ql.BlackConstantVol(calculation_date, calendar,volatility, day_count)
)

bsm_process = ql.BlackScholesMertonProcess(spot_price_handle, 
                                           dividend_ts_handle,
                                           yield_ts_handle,
                                           volatility_ts_handle)

In [0]:
time_steps = 1000
engine = ql.BinomialConvertibleEngine(bsm_process, "crr", time_steps)

In [33]:
convertible_bond.setPricingEngine(engine)
print ("NPV ", convertible_bond.NPV())

NPV  123.82876484726702
